## 1. 데이터 확인

### 필수 라이브러리

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore') # 경고 메세지 무시

In [3]:
df_train_origin = pd.read_csv("data/train.csv") # 학습용 데이터
df_test_origin = pd.read_csv("data/submission.csv") # 테스트 데이터(제출파일의 데이터)

In [4]:
df_train_origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59299 entries, 0 to 59298
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bant_submit              59299 non-null  float64
 1   customer_country         58317 non-null  object 
 2   business_unit            59299 non-null  object 
 3   com_reg_ver_win_rate     14568 non-null  float64
 4   customer_idx             59299 non-null  int64  
 5   customer_type            15338 non-null  object 
 6   enterprise               59299 non-null  object 
 7   historical_existing_cnt  13756 non-null  float64
 8   id_strategic_ver         3444 non-null   float64
 9   it_strategic_ver         1121 non-null   float64
 10  idit_strategic_ver       4565 non-null   float64
 11  customer_job             40566 non-null  object 
 12  lead_desc_length         59299 non-null  int64  
 13  inquiry_type             58358 non-null  object 
 14  product_category      

## 2. 데이터 전처리

### 각 변수별 확인

In [5]:
df_train_process = pd.read_csv('data/Ch2/df_train.csv')
df_test_process = pd.read_csv('data/Ch2/df_test.csv')

In [6]:
df_train_process.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59299 entries, 0 to 59298
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bant_submit              59299 non-null  float64
 1   customer_country         59299 non-null  object 
 2   business_unit            59299 non-null  object 
 3   com_reg_ver_win_rate     14568 non-null  float64
 4   customer_idx             59299 non-null  int64  
 5   customer_type            59299 non-null  object 
 6   enterprise               59299 non-null  object 
 7   historical_existing_cnt  59299 non-null  float64
 8   id_strategic_ver         59299 non-null  float64
 9   it_strategic_ver         59299 non-null  float64
 10  idit_strategic_ver       59299 non-null  float64
 11  customer_job             59299 non-null  object 
 12  lead_desc_length         59299 non-null  int64  
 13  inquiry_type             59299 non-null  object 
 14  product_category      

## 3. 피처엔지니어링

### 레이블 인코딩

In [7]:
df_train_encoded = pd.read_csv('data/Ch3/df_train.csv')
df_test_encoded = pd.read_csv('data/Ch3/df_test.csv')

In [8]:
df_train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59299 entries, 0 to 59298
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   bant_submit               59299 non-null  float64
 1   customer_country          59299 non-null  int64  
 2   business_unit             59299 non-null  int64  
 3   com_reg_ver_win_rate      59299 non-null  float64
 4   customer_idx              59299 non-null  int64  
 5   customer_type             59299 non-null  int64  
 6   enterprise                59299 non-null  int64  
 7   historical_existing_cnt   59299 non-null  float64
 8   customer_job              59299 non-null  int64  
 9   lead_desc_length          59299 non-null  int64  
 10  customer_country.1        59299 non-null  int64  
 11  customer_position         59299 non-null  int64  
 12  response_corporate        59299 non-null  int64  
 13  expected_timeline         59299 non-null  int64  
 14  lead_o

## 4. 모델 학습

### 데이터 분할

학습, 검증 데이터 분리

In [9]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(
    df_train_encoded.drop("is_converted", axis=1),
    df_train_encoded["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

### 모델 라이브러리

#### 단일모델 기준으로 사용할수 있는 모델들의 라이브러리

In [10]:
# from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np

# 보팅
from sklearn.ensemble import VotingClassifier
# 스테킹
from sklearn.ensemble import StackingClassifier

In [ ]:
break

### 최적 하이퍼 파라미터 찾기

#### optuna를 통한 최적의 파라미터 찾기

Random Forest

In [10]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np

def objectiveRF(trial, x_tr, y_tr, x_val, y_val):
    n_estimators = trial.suggest_int('n_estimators', 400, 1000)
    max_depth = trial.suggest_int('max_depth', 10, 35)
    min_samples_split = trial.suggest_int('min_samples_split', 10, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 8, 20)
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        criterion=criterion,
        random_state=0
    )

    model.fit(x_tr, y_tr)
    pred = model.predict(x_val)
    score = f1_score(y_val, pred, average="weighted")
    
    return score


# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=0))
study.optimize(lambda trial: objectiveRF(trial, x_train, y_train, x_val, y_val), n_trials=50)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

[I 2024-02-17 20:55:05,044] A new study created in memory with name: no-name-ba012bf8-a0a6-4cb2-840d-7020c281ceb5
[W 2024-02-17 20:55:09,497] Trial 0 failed with parameters: {'n_estimators': 729, 'max_depth': 28, 'min_samples_split': 16, 'min_samples_leaf': 15, 'criterion': 'entropy'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\HamIG\AppData\Local\Programs\Python\Python312\Lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\HamIG\AppData\Local\Temp\ipykernel_50360\4050503408.py", line 32, in <lambda>
    study.optimize(lambda trial: objectiveRF(trial, x_train, y_train, x_val, y_val), n_trials=50)
                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\HamIG\AppData\Local\Temp\ipykernel_50360\4050503408.py", line 23, in objectiveRF
    model.fit(x_tr, y_tr)
  File "c:\Users\HamIG\A

KeyboardInterrupt: 

LGBM

In [ ]:
import optuna
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

def objectiveLGBM(trial, x_tr, y_tr, x_val, y_val):
    param = {
        'num_leaves': trial.suggest_int('num_leaves', 2, 300),
        'max_depth': trial.suggest_int('max_depth', 2, 25),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'verbose' : -1,
        'random_state': 0
    }
    
    model = LGBMClassifier(**param)
    model.fit(x_tr, y_tr)
    pred = model.predict(x_val)
    score = f1_score(y_val, pred, average="weighted")
    
    return score


# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=0))
study.optimize(lambda trial: objectiveLGBM(trial, x_train, y_train, x_val, y_val), n_trials=1)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

[I 2024-02-17 17:24:10,277] A new study created in memory with name: no-name-5e1786de-1c89-42be-88aa-05fa97c761ec
[I 2024-02-17 17:24:12,914] Trial 0 finished with value: 0.9753345480842605 and parameters: {'num_leaves': 166, 'max_depth': 19, 'n_estimators': 643, 'learning_rate': 0.05903948646972072, 'min_child_samples': 24}. Best is trial 0 with value: 0.9753345480842605.


Best trial: score 0.9753345480842605, 
params {'num_leaves': 166, 'max_depth': 19, 'n_estimators': 643, 'learning_rate': 0.05903948646972072, 'min_child_samples': 24}


LGBM_dart

In [13]:
import optuna
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

def objectiveLGBM(trial, x_tr, y_tr, x_val, y_val):
    param = {
        'num_leaves': trial.suggest_int('num_leaves', 170, 350),
        'max_depth': trial.suggest_int('max_depth', 10, 30),
        'n_estimators': trial.suggest_int('n_estimators', 400, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.03, 0.1),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 20),
        'verbose' : -1,
        'boosting' : 'dart',
        'random_state': 0
    }
    
    model = LGBMClassifier(**param)
    model.fit(x_tr, y_tr)
    pred = model.predict(x_val)
    score = f1_score(y_val, pred, average="micro")
    
    return score


# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=0))
study.optimize(lambda trial: objectiveLGBM(trial, x_train, y_train, x_val, y_val), n_trials=100)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

[I 2024-02-18 13:11:33,910] A new study created in memory with name: no-name-7f3daecf-0bb9-4452-82cf-97d02256059a
[I 2024-02-18 13:12:05,034] Trial 0 finished with value: 0.9769814502529511 and parameters: {'num_leaves': 269, 'max_depth': 25, 'n_estimators': 762, 'learning_rate': 0.06814182280978279, 'min_child_samples': 11}. Best is trial 0 with value: 0.9769814502529511.
[I 2024-02-18 13:12:47,432] Trial 1 finished with value: 0.977150084317032 and parameters: {'num_leaves': 286, 'max_depth': 19, 'n_estimators': 935, 'learning_rate': 0.09745639323507206, 'min_child_samples': 11}. Best is trial 1 with value: 0.977150084317032.
[I 2024-02-18 13:13:12,508] Trial 2 finished with value: 0.9772344013490725 and parameters: {'num_leaves': 313, 'max_depth': 21, 'n_estimators': 741, 'learning_rate': 0.09479176468048628, 'min_child_samples': 6}. Best is trial 2 with value: 0.9772344013490725.
[I 2024-02-18 13:13:40,866] Trial 3 finished with value: 0.9776559865092749 and parameters: {'num_leave

Best trial: score 0.9784148397976391, 
params {'num_leaves': 170, 'max_depth': 13, 'n_estimators': 692, 'learning_rate': 0.0916736042020453, 'min_child_samples': 10}


XGBoost

In [15]:
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

def objectiveXGB(trial, x_tr, y_tr, x_val, y_val):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 1500, 2500),
        'learning_rate': trial.suggest_float('learning_rate', 0.03, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'objective': 'binary:logistic',  # 이진 분류
        'eval_metric': trial.suggest_categorical("eval_metric", ["logloss", "auc", "error"]),
        'random_state': 0
    }
    
    model = XGBClassifier(**param)
    model.fit(x_tr, y_tr)
    pred = model.predict(x_val)
    score = f1_score(y_val, pred, average="weighted")
    
    return score


# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=0))
study.optimize(lambda trial: objectiveXGB(trial, x_train, y_train, x_val, y_val), n_trials=100)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

[I 2024-02-17 22:01:44,818] A new study created in memory with name: no-name-f2ea9108-4035-4ae0-aaf6-0519e1ab98eb
[I 2024-02-17 22:01:50,560] Trial 0 finished with value: 0.9741781813668445 and parameters: {'n_estimators': 2049, 'learning_rate': 0.08006325564606936, 'max_depth': 10, 'eval_metric': 'error'}. Best is trial 0 with value: 0.9741781813668445.
[I 2024-02-17 22:01:56,881] Trial 1 finished with value: 0.9743102014356569 and parameters: {'n_estimators': 1938, 'learning_rate': 0.09242411005474559, 'max_depth': 15, 'eval_metric': 'auc'}. Best is trial 1 with value: 0.9743102014356569.
[I 2024-02-17 22:01:59,313] Trial 2 finished with value: 0.9691065698382101 and parameters: {'n_estimators': 2068, 'learning_rate': 0.09479176468048628, 'max_depth': 3, 'eval_metric': 'error'}. Best is trial 1 with value: 0.9743102014356569.
[I 2024-02-17 22:02:06,775] Trial 3 finished with value: 0.9742829019304714 and parameters: {'n_estimators': 2278, 'learning_rate': 0.09090085037727735, 'max_de

Best trial: score 0.9752682275927347, 
params {'n_estimators': 2456, 'learning_rate': 0.08433865198013414, 'max_depth': 7, 'eval_metric': 'auc'}


Decision Tree

In [ ]:
import optuna
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

def objectiveDT(trial, x_tr, y_tr, x_val, y_val):
    param = {
        'max_depth': trial.suggest_int('max_depth', 5, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 50),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'random_state': 0
    }
    
    model = DecisionTreeClassifier(**param)
    model.fit(x_tr, y_tr)
    pred = model.predict(x_val)
    score = f1_score(y_val, pred, average="weighted")
    
    return score


# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=0))
study.optimize(lambda trial: objectiveDT(trial, x_train, y_train, x_val, y_val), n_trials=1)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

[I 2024-02-17 17:24:29,037] A new study created in memory with name: no-name-a651a7a3-95ce-4a8a-84f8-6916ff40f76a
[I 2024-02-17 17:24:29,257] Trial 0 finished with value: 0.9615427574699139 and parameters: {'max_depth': 30, 'min_samples_split': 37, 'min_samples_leaf': 13, 'criterion': 'gini'}. Best is trial 0 with value: 0.9615427574699139.


Best trial: score 0.9615427574699139, 
params {'max_depth': 30, 'min_samples_split': 37, 'min_samples_leaf': 13, 'criterion': 'gini'}


ExtraTrees

In [ ]:
import optuna
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

def objectiveET(trial, x_tr, y_tr, x_val, y_val):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 400, 1000),
        'max_depth': trial.suggest_int('max_depth', 10, 35),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'random_state': 0
    }
    
    model = ExtraTreesClassifier(**param)
    model.fit(x_tr, y_tr)
    pred = model.predict(x_val)
    score = f1_score(y_val, pred, average="weighted")
    
    return score


# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=0))
study.optimize(lambda trial: objectiveET(trial, x_train, y_train, x_val, y_val), n_trials=1)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

[I 2024-02-17 17:24:34,746] A new study created in memory with name: no-name-4aca2d4a-d2ce-4e08-b4e7-56f635543340
[I 2024-02-17 17:24:48,227] Trial 0 finished with value: 0.922567475656384 and parameters: {'n_estimators': 729, 'max_depth': 28, 'min_samples_split': 13, 'min_samples_leaf': 6, 'criterion': 'entropy'}. Best is trial 0 with value: 0.922567475656384.


Best trial: score 0.922567475656384, 
params {'n_estimators': 729, 'max_depth': 28, 'min_samples_split': 13, 'min_samples_leaf': 6, 'criterion': 'entropy'}


GradientBoost

In [ ]:
import optuna
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

def objectiveGB(trial, x_tr, y_tr, x_val, y_val):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 700, 1500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'max_depth': trial.suggest_int('max_depth', 2, 15),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
        'random_state': 0
    }
    
    model = GradientBoostingClassifier(**param)
    model.fit(x_tr, y_tr)
    pred = model.predict(x_val)
    score = f1_score(y_val, pred, average="weighted")
    
    return score


# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=0))
study.optimize(lambda trial: objectiveGB(trial, x_train, y_train, x_val, y_val), n_trials=1)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))


[I 2024-02-17 17:26:44,855] A new study created in memory with name: no-name-a1d289c3-67d6-4220-b852-49ba548f1867
[I 2024-02-17 17:30:06,455] Trial 0 finished with value: 0.9773593142990754 and parameters: {'n_estimators': 1139, 'learning_rate': 0.07436704297351776, 'max_depth': 10, 'min_samples_leaf': 11}. Best is trial 0 with value: 0.9773593142990754.


Best trial: score 0.9773593142990754, 
params {'n_estimators': 1139, 'learning_rate': 0.07436704297351776, 'max_depth': 10, 'min_samples_leaf': 11}


AdaBoost

In [15]:
import optuna
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

def objectiveAdaBoost(trial, x_tr, y_tr, x_val, y_val):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 1000, 2000),
        'learning_rate': trial.suggest_float('learning_rate', 0.1, 1.0),
        'algorithm': trial.suggest_categorical('algorithm', ['SAMME.R']),
        'random_state': 0
    }
    
    model = AdaBoostClassifier(**param)
    model.fit(x_tr, y_tr)
    pred = model.predict(x_val)
    score = f1_score(y_val, pred, average="weighted")
    
    return score


# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=0))
study.optimize(lambda trial: objectiveAdaBoost(trial, x_train, y_train, x_val, y_val), n_trials=80)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

[I 2024-02-18 14:18:46,854] A new study created in memory with name: no-name-8e7f0449-1032-442b-8368-47e588a750a5
[I 2024-02-18 14:19:48,953] Trial 0 finished with value: 0.9606806529165666 and parameters: {'n_estimators': 1549, 'learning_rate': 0.7436704297351775, 'algorithm': 'SAMME.R'}. Best is trial 0 with value: 0.9606806529165666.
[I 2024-02-18 14:20:52,378] Trial 1 finished with value: 0.9599116042694162 and parameters: {'n_estimators': 1603, 'learning_rate': 0.5903948646972071, 'algorithm': 'SAMME.R'}. Best is trial 0 with value: 0.9606806529165666.
[I 2024-02-18 14:21:48,775] Trial 2 finished with value: 0.9609638384546466 and parameters: {'n_estimators': 1424, 'learning_rate': 0.6813047017599905, 'algorithm': 'SAMME.R'}. Best is trial 2 with value: 0.9609638384546466.
[I 2024-02-18 14:22:38,742] Trial 3 finished with value: 0.9616384187385358 and parameters: {'n_estimators': 1438, 'learning_rate': 0.9025957007038717, 'algorithm': 'SAMME.R'}. Best is trial 3 with value: 0.9616

KeyboardInterrupt: 

### 모델 정의

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np

# 보팅
from sklearn.ensemble import VotingClassifier
# 스테킹
from sklearn.ensemble import StackingClassifier

In [28]:
# RandomForest
rf_model = RandomForestClassifier(
    n_estimators=1056     
    , max_depth=30    
    , min_samples_split=3   
    , min_samples_leaf=1   
    , bootstrap=True
    , criterion='entropy'
)

# LightGBM
lgb_model = LGBMClassifier(
    num_leaves=162
    , max_depth=10
    , n_estimators=487
    , learning_rate=0.07324658507873466
    , min_child_samples=31
    , verbose = -1
)

# LightGBM_dart
lgb_dart_model = LGBMClassifier(
    num_leaves=170
    , max_depth=13
    , n_estimators=692
    , learning_rate=0.0916736042020453
    , min_child_samples=10
    , verbose = -1
    , boosting_type="dart"
)

# XGBoost 
xgb_model = XGBClassifier(
    n_estimators=1427
    , learning_rate=0.08645845446703926
    , max_depth=7
    , objective='binary:logistic'
    , eval_metric = 'error'
)

# GradientBoosting
gb_model = GradientBoostingClassifier(
    n_estimators=1425
    , learning_rate=0.09883679411048218
    , max_depth=6
    , min_samples_leaf=13
)

# DecisionTree
dt_model = DecisionTreeClassifier(
    max_depth=24
    , min_samples_split=2  # 노드를 분할하기 위한 최소 샘플 수
    , min_samples_leaf=1  # 리프 노드에 필요한 최소 샘플 수
    , criterion = 'entropy'
)  

# ExtraTrees
et_model = ExtraTreesClassifier(
    n_estimators=486
    , min_samples_split=3  # 노드를 분할하기 위한 최소 샘플 수
    , min_samples_leaf=1   # 리프 노드에 필요한 최소 샘플 수
    , max_depth=26 
    , criterion = 'entropy'
)  

# AdaBoost
ada_model = AdaBoostClassifier(
    n_estimators=1399
    , learning_rate=0.9987147599335517
    , algorithm='SAMME.R'
)  

# CatBoost
#cat_model = CatBoostClassifier(
#    iterations=1045
#   , learning_rate=0.21147352826666405
#    , depth=9
#    , verbose=False
#)


### 스태킹 분류기 생성 ###
model = StackingClassifier(
    estimators=[
        ('rf', rf_model),
        ('lgb', lgb_model),
        ('xgb', xgb_model),
        ('gb', gb_model),
        ('dt', dt_model),
        ('lgb_dart',lgb_dart_model)
        #('cat', cat_model)
    ],
    final_estimator=lgb_model  # 최종 메타 모델
)



### 모델 학습

In [29]:
model.fit(x_train, y_train)

StackingClassifier(estimators=[('rf',
                                RandomForestClassifier(criterion='entropy',
                                                       max_depth=30,
                                                       min_samples_split=3,
                                                       n_estimators=1056)),
                               ('lgb',
                                LGBMClassifier(learning_rate=0.07324658507873466,
                                               max_depth=10,
                                               min_child_samples=31,
                                               n_estimators=487, num_leaves=162,
                                               verbose=-1)),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel...
                                DecisionTreeClassifier(criterion='entropy',
                                                       max_depth=24)),
                               ('lgb_dart',
                                LGBMClassifier(boosting_type='dart',
                                               learning_rate=0.0916736042020453,
                                               max_depth=13,
                                               min_child_samples=10,
                                               n_estimators=692, num_leaves=170,
                                               verbose=-1))],
                   final_estimator=LGBMClassifier(boosting_type='dart',
                                                  learning_rate=0.0916736042020453,
                                                  max_depth=13,
                                                  min_child_samples=10,
                                                  n_estimators=692,
                                                  num_leaves=170, verbose=-1))

### 모델 성능 보기

In [30]:
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])
    weighted_F1 = f1_score(y_test, y_pred, average='weighted')  # 추가된 부분 

    metrics = pd.DataFrame({
        '정확도': [accuracy],
        '정밀도': [precision],
        '재현율': [recall],
        'F1 Score': [F1],
        'Weighted F1': [weighted_F1]  # 추가된 부분
    })

    confusion_df = pd.DataFrame(confusion, index=['True', 'False'], columns=['True', 'False'])

    print("\n오차행렬:")
    display(confusion_df)
    print("평가 지표:")
    display(metrics)


In [31]:
pred = model.predict(x_val)
get_clf_eval(y_val, pred)


오차행렬:


,True,False
True,756,191
False,101,10812


평가 지표:


,정확도,정밀도,재현율,F1 Score,Weighted F1
0,0.975379,0.882147,0.79831,0.838137,0.974816


## 4. 제출하기

### 테스트 데이터 예측

In [32]:
# 예측에 필요한 데이터 분리
x_test = df_test_encoded.drop(["is_converted", "id"], axis=1)

In [33]:
test_pred = model.predict(x_test)
sum(test_pred) # True로 예측된 개수

625

### 제출 파일 작성

In [ ]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("./data/submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission_model_12.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**

.